# TODO

~~1. Generate a few keywords for each starting game~~

~~2. Write a guesser agent to guess the keyword after each answer~~

~~3. Generate a JSON list of all the games~~

In [1]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [244]:
class DecisionNode:
    def __init__(self, question=None, yes_branch=None, no_branch=None, examples=None, summary = None):
        self.question = question
        self.yes_branch = yes_branch
        self.no_branch = no_branch
        self.examples = examples
        self.summary = summary

def print_tree(node, indent=0):
    """ Recursively print the decision tree. """
    if node.summary:
        # Print leaf node examples
        print(' ' * indent + f'→ Summary: The keyword is {node.summary}.')
    if node.examples:
        # Print leaf node examples
        print(' ' * indent + f'→ Examples: {", ".join(node.examples)}')
    else:
        # Print the question node
        turn = indent // 4 + 1
        print(' ' * indent + f'Turn {turn}: {node.question}')
        if node.yes_branch or node.no_branch:
            print(' ' * indent + ' ├─ Yes:')
            print_tree(node.yes_branch, indent + 4)  # Print yes branch
            print(' ' * indent + ' └─ No:')
            print_tree(node.no_branch, indent + 4)   # Print no branch

# Define the decision tree structure
tree = DecisionNode(
    "Is it a place?",
    yes_branch=DecisionNode(
        "Is it a country?",
        yes_branch=DecisionNode(
            "Is this country either in Asia or in Africa?",
            yes_branch=DecisionNode(
                "Is this country in Asia?",
                yes_branch=DecisionNode(summary='a country in Asia', examples=['Kazakhstan', 'South Korea']),
                no_branch=DecisionNode(summary='a country in Africa', examples=['Morocco', 'Ghana'])
            ),
            no_branch=DecisionNode(
                "Is this country in Europe?",
                yes_branch=DecisionNode(summary='a country in Europe', examples=['Romania', 'Germany']),
                no_branch=DecisionNode(
                    "Is this country in the Americas?",
                    yes_branch=DecisionNode(summary='a country in the Americas', examples=['Argentina', 'Canada']),
                    no_branch=DecisionNode(summary='a country in Oceania', examples=['New Zealand', 'Samoa'])
                )
            )
        ),
        no_branch=DecisionNode(
            "Is it a city?",
            yes_branch=DecisionNode(
                "Is it a capital city?",
                yes_branch=DecisionNode(summary='a capital city', examples=['Moscow', 'Santiago']),
                no_branch=DecisionNode(summary='a city which is not a capital', examples=['Chicago', 'Tianjin'])
            ),
            no_branch=DecisionNode(examples=["Park", "Stadium"])
        )
    ),
    no_branch=DecisionNode(
        "Is it a man-made thing?",
        yes_branch=DecisionNode(
            "Is it something found indoors?",
            yes_branch=DecisionNode(
                "Is it something found in a home?",
                yes_branch=DecisionNode(summary='a man-made thing found in people\'s homes', examples=["Bed frame", "Stove"]),
                no_branch=DecisionNode(summary='a man-made thing not found in people\'s homes', examples=["Hospital bed", "Erlenmeyer flask"])
            ),
            no_branch=DecisionNode(summary='a man-made thing found outdoors', examples=["School", "Fence"])
        ),
        no_branch=DecisionNode(
            "Is it a living thing?",
            yes_branch=DecisionNode(
                "Is it an animal?",
                yes_branch=DecisionNode(summary='an animal', examples=["Elephant", "Sea Urchin"]),
                no_branch=DecisionNode(
                    "Is it a plant?",
                    yes_branch=DecisionNode(summary='a plant', examples=["Oak tree", "Cactus"]),
                    no_branch=DecisionNode(summary='a fungus or bacteria', examples=['E Coli', 'Wood ear mushroom'])
                )
            ),
            no_branch=DecisionNode(
                "Is it a geological ?",
                yes_branch=DecisionNode(examples=["Gravity", "Evolution"]),
                no_branch=DecisionNode(examples=["Happiness", "Freedom"])
            )
        )
    )
)

# Print the decision tree
print_tree(tree)

Turn 1: Is it a place?
 ├─ Yes:
    Turn 2: Is it a country?
     ├─ Yes:
        Turn 3: Is this country either in Asia or in Africa?
         ├─ Yes:
            Turn 4: Is this country in Asia?
             ├─ Yes:
                → Summary: The keyword is a country in Asia.
                → Examples: Kazakhstan, South Korea
             └─ No:
                → Summary: The keyword is a country in Africa.
                → Examples: Morocco, Ghana
         └─ No:
            Turn 4: Is this country in Europe?
             ├─ Yes:
                → Summary: The keyword is a country in Europe.
                → Examples: Romania, Germany
             └─ No:
                Turn 5: Is this country in the Americas?
                 ├─ Yes:
                    → Summary: The keyword is a country in the Americas.
                    → Examples: Argentina, Canada
                 └─ No:
                    → Summary: The keyword is a country in Oceania.
                    → Examples: Ne

In [243]:
def generate_games_for_leaves(node, path=None, games=None):
    if path is None:
        path = []
    if games is None:
        games = []

    if node.examples or node.summary:
        # We've reached a leaf node; append the game
        games.append(path.copy())

    if node.yes_branch:
        path.append({"role": "questioner", "content": node.question})
        path.append({"role": "answerer", "content": "Yes"})
        generate_games_for_leaves(node.yes_branch, path, games)
        # Backtrack
        path.pop()
        path.pop()

    if node.no_branch:
        path.append({"role": "questioner", "content": node.question})
        path.append({"role": "answerer", "content": "No"})
        generate_games_for_leaves(node.no_branch, path, games)
        # Backtrack
        path.pop()
        path.pop()

    return games

def game_to_string(game):
    return '\n'.join([f'{step["role"]}: {step["content"]}' for step in game])

def print_game(game):
    print(game_to_string(game))

def print_games(games):
    for x, game in enumerate(games):
        print(x)
        print_game(game)
        print()  # Newline between games

# Generate games for the provided decision tree
games = generate_games_for_leaves(tree)

# Print out the games
print_games(games)

0
questioner: Is it a place?
answerer: Yes
questioner: Is it a country?
answerer: Yes
questioner: Is this country either in Asia or in Africa?
answerer: Yes
questioner: Is this country in Asia?
answerer: Yes

1
questioner: Is it a place?
answerer: Yes
questioner: Is it a country?
answerer: Yes
questioner: Is this country either in Asia or in Africa?
answerer: Yes
questioner: Is this country in Asia?
answerer: No

2
questioner: Is it a place?
answerer: Yes
questioner: Is it a country?
answerer: Yes
questioner: Is this country either in Asia or in Africa?
answerer: No
questioner: Is this country in Europe?
answerer: Yes

3
questioner: Is it a place?
answerer: Yes
questioner: Is it a country?
answerer: Yes
questioner: Is this country either in Asia or in Africa?
answerer: No
questioner: Is this country in Europe?
answerer: No
questioner: Is this country in the Americas?
answerer: Yes

4
questioner: Is it a place?
answerer: Yes
questioner: Is it a country?
answerer: Yes
questioner: Is this

In [249]:
def construct_messages(game, role, keyword = None, guesses = []):
    if role == "questioner":
        prompt = {
            "role": "system",
            "content": "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific place or thing. You are participating in a new game of 20 Questions. Your role is to be the Questioner. You will ask successive yes-or-no questions to determine the keyword. You have a limited number of questions to ask, so choose a question that will eliminate half of the possible keywords to maximize efficiency. Avoid asking questions that are too specific too early on. Be as vague as possible while still eliminating half of the remaining possibilities. DO NOT try to guess the specific keyword, leave that to the guesser. Try to ask the full 20 questions. If you accidentally guessed correctly though, simply output [GAME OVER]. Do not output any text other than the question.",
        }
        messages = [prompt]
        for message in game:
            if message["role"] == "questioner":
                messages.append({
                    "role": "assistant",
                    "content": message["content"]
                })
            else:
                messages.append({
                    "role": "user",
                    "content": message["content"]
                })
    elif role == "answerer":
        prompt = {
            "role": "system",
            "content": f"You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific place or thing. You are participating in a new game of 20 Questions. Your role is to be the Answerer. The keyword is {keyword}. Answer only Yes or No based on the keyword. Do not output any other text.",
        }
        messages = [prompt]
        for message in game:
            if message["role"] == "questioner":
                messages.append({
                    "role": "user",
                    "content": message["content"]
                })
            else:
                messages.append({
                    "role": "assistant",
                    "content": message["content"]
                })
    elif role == "guesser":
        prompt = {
            "role": "system",
            "content": "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific place or thing. You are participating in a new game of 20 Questions. Your role is to be the Guesser. Based on the given questions and answers, guess the keyword at this point. Do not ask a question, just state the guessed keyword with no other text except the keyword itself. DO NOT output any other text other than the guessed keyword.", #The game will end immediately when you guess correctly, so never repeat a guess. 
        }
        messages = [prompt, {
            "role": "user",
            "content": game_to_string(game) + "\nYour guess: "# + "\nPrevious guesses:\n" + '\n'.join(guesses)
        }]
    return messages

construct_messages(games[3], "guesser", "", guesses=["a", "b"])

[{'role': 'system',
  'content': 'You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific place or thing. You are participating in a new game of 20 Questions. Your role is to be the Guesser. Based on the given questions and answers, guess the keyword at this point. Do not ask a question, just state the guessed keyword with no other text except the keyword itself. DO NOT output any other text other than the guessed keyword.'},
 {'role': 'user',
  'content': 'questioner: Is it a place?\nanswerer: Yes\nquestioner: Is it a country?\nanswerer: Yes\nquestioner: Is this country either in Asia or in Africa?\nanswerer: No\nquestioner: Is this country in Europe?\nanswerer: No\nquestioner: Is this country in the Americas?

In [248]:
import json
def generate_possible_keywords(game, num_keywords = 10):
    def inner():
        prompt = {
                "role": "system",
                "content": f"You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific place or thing. You are helping set up the game by coming up with potential keywords that fit the start of a game. Given the start of a game, give a list of {num_keywords} that could be answers to the game. Output the keywords in a JSON array of strings E.g. [\"apple\", ...]. Do NOT output any other text besides the array.",
            }
        messages = [prompt, {
                "role": "user",
                "content": game_to_string(game) + "\nKeywords: "
            }]
        result = client.chat.completions.create(
                messages=messages,
                model="gpt-4o",
            ).choices[0].message.content
        return result
    for attempt in range(5):
        result = json.loads(inner())
        if len(result) < 10:
            print("Too few results")
            continue
        elif len(result) > 10:
            return result[:10]
        return result

generate_possible_keywords(games[9])

['printer',
 'desk',
 'chair',
 'whiteboard',
 'projector',
 'vending machine',
 'elevator',
 'filing cabinet',
 'conference table',
 'fire extinguisher']

In [204]:
def simulate_game(starting_game, keyword):
    game = starting_game[:]

    starting_game_length = int(len(starting_game) / 2)

    questions = [step["content"] for step in game if step["role"] == "questioner"]
    answers = [step["content"] for step in game if step["role"] == "answerer"]
    guesses = []

    guesser_finished = False

    for i in range(0,starting_game_length * 2, 2):
        slice = game[i:i+2]
        guess = client.chat.completions.create(
            messages=construct_messages(slice, "guesser", keyword),
            model="gpt-4o",
        ).choices[0].message.content

        guesses.append(guess)

        print("questioner: " + slice[0]["content"])
        print("answerer: " + slice[1]["content"])
        print(f"guesser: {guess}\n")

    for _ in range(20 - starting_game_length):
        question = client.chat.completions.create(
            messages=construct_messages(game, "questioner", keyword),
            model="gpt-4o",
        ).choices[0].message.content
    
        game.append({
            "role": "questioner",
            "content": question
        })

        print(f"questioner: {question}")

        if "OVER" in question:
            break
        
        questions.append(question)
    
        answer = client.chat.completions.create(
            messages=construct_messages(game, "answerer", keyword),
            model="gpt-4o",
        ).choices[0].message.content
    
        game.append({
            "role": "answerer",
            "content": answer
        })

        answers.append(answer)
        print(f"answerer: {answer}")
        
        guess = client.chat.completions.create(
            messages=construct_messages(game, "guesser", keyword),
            model="gpt-4o",
        ).choices[0].message.content

        guesses.append(guess)
        print(f"guesser: {guess}\n")

    return {
        "keyword": keyword,
        "questions": questions,
        "answers": answers,
        "guesses": guesses
    }

result = simulate_game(games[8], "folder")
print(result)

questioner: Is it a place?
answerer: No
guesser: thing

questioner: Is it a man-made thing?
answerer: Yes
guesser: Clock

questioner: Is it something found indoors?
answerer: Yes
guesser: Chair

questioner: Is it something found in a home?
answerer: Yes
guesser: Table

questioner: Is it an electronic device?
answerer: No
guesser: Chair

questioner: Is it typically found in the kitchen?
answerer: No
guesser: Chair

questioner: Is it used for entertainment or recreation?
answerer: No
guesser: Chair

questioner: Is it a piece of furniture?
answerer: No
guesser: Book

questioner: Is it something you use daily?
answerer: Yes
guesser: Toothbrush

questioner: Is it smaller than a microwave oven?
answerer: Yes
guesser: Toothbrush

questioner: Is it something that you wear?
answerer: No
guesser: Toothbrush

questioner: Is it used for personal hygiene?
answerer: No
guesser: Toothbrush

questioner: Is it used for work or productivity?
answerer: Yes
guesser: Pen

questioner: Is it commonly found i

## Generate all the games

In [230]:
game_results = []
for g_index,g in enumerate(games):
    print("Game:", g_index)
    generated_keywords = generate_possible_keywords(g, num_keywords=10)
    for generated_keyword in generated_keywords:
        print("Generated keyword:", generated_keyword)
        game_result = simulate_game(g, generated_keyword)
        game_results.append(game_result)

Game: 0
Generated keyword: Japan
questioner: Is it a place?
answerer: Yes
guesser: Paris

questioner: Is it a country?
answerer: Yes
guesser: France

questioner: Is this country either in Asia or in Africa?
answerer: Yes
guesser: India

questioner: Is this country in Asia?
answerer: Yes
guesser: China

questioner: Is this country located in East Asia?
answerer: Yes
guesser: China

questioner: Is this country an island nation?
answerer: Yes
guesser: Japan

questioner: Is this country part of the United Nations?
answerer: Yes
guesser: Japan

questioner: Is this country Japan?
answerer: Yes
guesser: Japan

questioner: [GAME OVER]
Generated keyword: China
questioner: Is it a place?
answerer: Yes
guesser: Park

questioner: Is it a country?
answerer: Yes
guesser: Japan

questioner: Is this country either in Asia or in Africa?
answerer: Yes
guesser: India

questioner: Is this country in Asia?
answerer: Yes
guesser: China

questioner: Is it located in East Asia?
answerer: Yes
guesser: China

q

In [234]:
len(game_results)

160

## Save as JSONL

In [242]:
with open("games-1.jsonl", "w") as f:
    for i in game_results:
        json.dump(i, f)
        f.write('\n')